<center> بسم الله الرحمن الرحيم </center>

# **Solving Sudoku with CSP**
## Project description:
The aim of this project is to be able to represent the Sudoku puzzle as a constraint satisfaction problem (CSP) and solve it by applying: (1) Standard backtracking (BT), (2) Forward Checking algorithm (FC), and (3) Maintaining Arc-Consistency algorithm (MAC). You are free to use any additional heuristics.
## Understanding Sudoku and Constraint Satisfaction Problems (CSPs):
Sudoku is a logic-based number placement puzzle that has gained worldwide popularity. The objective of the game is to fill a 9×9 grid with digits so that each column, each row, and each of the nine 3×3 subgrids that compose the grid contain all of the digits from 1 to 9. The puzzle setter provides a partially completed grid, which typically has a unique solution.
A Constraint Satisfaction Problem (CSP) is a mathematical problem defined as a set of objects whose state must satisfy a number of constraints or restrictions. CSPs represent the entities in a problem as a homogeneous collection of finite constraints over variables, which is solved by constraint satisfaction methods. CSPs are a powerful concept for expressing and solving problems in various areas, such as artificial intelligence, computer science, and operations research.

<br>

<center>
<table>
  <tr>
    <td><img src="img/sudoku_before.png" alt="Sudoku Puzzle before solving"></td>
    <td><img src="img/sudoku_after.png" alt="Sudoku Puzzle after solving"></td>
  </tr>
  <tr>
    <td style="text-align: center;">Sudoku Puzzle before solving</td>
    <td style="text-align: center;">Sudoku Puzzle after solving</td>
  </tr>
</table>
</center>




<br>

### Formulating Sudoku as a CSP involves the following steps:
   1. **Variables**
   Each cell in the Sudoku grid is considered as a variable. This gives us 81 variables 
   (9 rows × 9 columns), each with a domain of {1,2,…,9}.
   2. **Domains**
   The domain of each variable (cell) is the set of numbers from 1 to 9.
   3. **Constrains**
    The constraints are based on the rules of Sudoku:
        * All numbers in each row must be different.
        * All numbers in each 3×3 grid must be different.
        * If the Sudoku puzzle is partially completed, there are additional constraints for each cell that contains a number.

  By formulating Sudoku as a CSP, we can apply various algorithms used for solving CSPs to solve Sudoku puzzles.        

## Loading the dataset

In [ ]:
import pandas as pd

# Load the dataset
data = pd.read_csv('sudoku.csv')

# Get the puzzles and solutions
puzzles = data['quizzes']
solutions = data['solutions']


## Solve by using backtracking

In [ ]:
def solve(puzzle):
    find = find_empty(puzzle)
    if not find:
        return True
    else:
        row, col = find

    for i in range(1,10):
        if valid(puzzle, i, (row, col)):
            puzzle[row][col] = i

            if solve(puzzle):
                return True

            puzzle[row][col] = 0

    return False

def find_empty(puzzle):
    for i in range(len(puzzle)):
        for j in range(len(puzzle[0])):
            if puzzle[i][j] == 0:
                return (i, j)  # row, col

    return None

def valid(puzzle, num, pos):
    # Check row
    for i in range(len(puzzle[0])):
        if puzzle[pos[0]][i] == num and pos[1] != i:
            return False

    # Check column
    for i in range(len(puzzle)):
        if puzzle[i][pos[1]] == num and pos[0] != i:
            return False

    # Check box
    box_x = pos[1] // 3
    box_y = pos[0] // 3

    for i in range(box_y*3, box_y*3 + 3):
        for j in range(box_x * 3, box_x*3 + 3):
            if puzzle[i][j] == num and (i,j) != pos:
                return False

    return True


## Solve by forward-checking

In [6]:
def forward_checking(puzzle):
    # Find an unassigned cell
    unassigned = find_unassigned(puzzle)
    if not unassigned:
        return True  # Puzzle solved

    row, col = unassigned

    # Try all numbers from 1 to 9
    for num in range(1, 10):
        # Check feasibility of num at pos (row, col)
        if is_feasible(puzzle, row, col, num):
            # Make tentative assignment
            puzzle[row][col] = num

            # Return true if success, else undo the assignment and try again
            if forward_checking(puzzle):
                return True

            puzzle[row][col] = 0

    return False

# This function iterates over each cell in the puzzle until it finds an unassigned cell (a cell with the value 0), and then it returns the position of that cell.
def find_unassigned(puzzle):
    for i in range(9):
        for j in range(9):
            if puzzle[i][j] == 0:
                return (i, j)  # row, col

    return None

def is_feasible(puzzle, row, col, num):
    # Check the row
    for x in range(9):
        if puzzle[row][x] == num:
            return False

    # Check the column
    for x in range(9):
        if puzzle[x][col] == num:
            return False

    # Check the box
    start_row = row - row % 3
    start_col = col - col % 3
    for i in range(3):
        for j in range(3):
            if puzzle[i + start_row][j + start_col] == num:
                return False

    return True


## Now using MAC

In [ ]:
def mac(puzzle):
    # Find an unassigned cell
    unassigned = find_unassigned(puzzle)
    if not unassigned:
        return True  # Puzzle solved

    row, col = unassigned

    # Try all numbers from 1 to 9
    for num in range(1, 10):
        # Check feasibility of num at pos (row, col)
        if is_feasible(puzzle, row, col, num):
            # Make tentative assignment
            puzzle[row][col] = num

            # Invoke AC-3 algorithm to maintain arc consistency after assignment
            if ac3(puzzle):
                # If success, proceed with next unassigned variable
                if mac(puzzle):
                    return True

            # Undo the assignment
            puzzle[row][col] = 0

    return False

def ac3(puzzle):
    # Initialize a queue with all the arcs in the puzzle
    queue = [(i, j) for i in range(9) for j in range(9) if puzzle[i][j] == 0]

    while queue:
        i, j = queue.pop(0)

        # If revise(puzzle, i, j) returns True, then we add all neighboring cells to the queue
        if revise(puzzle, i, j):
            if not puzzle[i][j]:
                return False

            for h in get_neighbors(i, j):
                queue.append(h)

    return True

def revise(puzzle, i, j):
    revised = False
    for val in puzzle[i][j]:
        if not any(is_feasible(puzzle, i, j, val)):
            puzzle[i][j].remove(val)
            revised = True
    return revised

def get_neighbors(i, j):
    neighbors = []

    # Get row neighbors
    for x in range(9):
        if x != j:
            neighbors.append((i, x))

    # Get column neighbors
    for x in range(9):
        if x != i:
            neighbors.append((x, j))

    # Get box neighbors
    start_row, start_col = 3 * (i // 3), 3 * (j // 3)
    for x in range(start_row, start_row + 3):
        for y in range(start_col, start_col + 3):
            if x != i and y != j:
                neighbors.append((x, y))

    return neighbors
